In [11]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from utility import calculate_input_gradients, perturb_inputs, preprocess_images, \
                    postprocess_features, save_data_hdf5,get_dataset_hdf5,\
                    build_one_class_svm, combine_inliners_outliers, apply_temp_scale_to_model,\
                    apply_log_temp_scale_to_model, perturb_inputs_odin, extract_layer_features
 
from utility_db_outliers import load_dataset
from models_lib import load_custom_model_for_ds
import h5py
# from metrics2 import *
from metrics import *
from general_setting import *
import time
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
#------------
from utility_methods import *
# from utility_methods2 import *
from sklearn.metrics import roc_auc_score

from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import scipy.io as sio
import tensorflow.keras.backend as K
import math

In [2]:
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten, Lambda
from tensorflow.keras.layers import Reshape, Conv2DTranspose, MaxPooling2D, UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [4]:
BATCH_SIZE = 100
SAVE_RESULTS = True
id_name=ID_DS_LIST[1] # selects the ID dataset.
id_model=ID_MODEL_LIST[1]  # select the deep model used for training ID dataset.
print(id_name,id_model)

CIFAR10 VGG


In [5]:
(org_traing_data, org_training_labels),(id_eva_data,org_testing_labels)  = load_dataset(id_name)

org_traing_data_processed = preprocess_images(id_name, org_traing_data, id_model, verbose=True)
id_eva_data_processed = preprocess_images(id_name, id_eva_data, id_model, verbose=True)

Preprocessing was done for  CIFAR10 VGG
Preprocessing was done for  CIFAR10 VGG


In [6]:
x_train = org_traing_data_processed
y_train = org_training_labels
x_test = id_eva_data_processed
y_test = org_testing_labels

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler( )
scaler.fit(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test_0 = scaler.transform(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test = x_test_0.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],x_test.shape[3])

In [8]:
scaler.fit(x_train.flatten().reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]*x_train.shape[3]))
x_train_0 = scaler.transform(x_train.flatten().reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]*x_train.shape[3]))
x_train = x_train_0.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],x_train.shape[3])

In [12]:
# 网络参数
input_shape = (32, 32, 3)
batch_size = 256
kernel_size = 3
# filters = 16  
latent_dim = 1024 # 隐变量取2维只是为了方便后面画图
epochs = 100
x_in = Input(shape=input_shape)
x = x_in

In [13]:
structure = [32, 64, 128]

In [14]:
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                activation='relu',
                strides=2,
                padding='same')(x)
    # elif filters == "max":
    #     x = MaxPooling2D((2, 2), padding="same")(x)
    # elif filters == "average":
    #     x = AveragePooling2D((2, 2), padding="same")(x)

x = Flatten()(x)
x = Dense(latent_dim)(x)


In [15]:
x = Reshape((4,4,64))(x)
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2DTranspose(filters=filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=2,
                    padding='same')(x)
    # elif filters == "max" or filters == "average":
    #             x = UpSampling2D((2, 2))(x)

decoded = Conv2DTranspose(filters=3,
                        kernel_size=kernel_size,
                        activation='sigmoid',
                        padding='same')(x)

In [16]:
# 建立模型
ae = Model(x_in, decoded)

In [11]:
# 搭建为一个独立的模型
ae.compile(loss='mean_squared_error',optimizer='rmsprop')

In [13]:
ae.fit(x_train,x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)

Epoch 1/100
50000/50000 [==============================] - 6s 126us/step - loss: 2.3615
Epoch 2/100
50000/50000 [==============================] - 6s 127us/step - loss: 2.1462
Epoch 3/100
50000/50000 [==============================] - 6s 124us/step - loss: 1.9788
Epoch 4/100
50000/50000 [==============================] - 6s 122us/step - loss: 1.8401
Epoch 5/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.7199
Epoch 6/100
50000/50000 [==============================] - 6s 124us/step - loss: 1.6196
Epoch 7/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.5336
Epoch 8/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.4645
Epoch 9/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.3971
Epoch 10/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.3383
Epoch 11/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.2868
Epoch 12/100
50000/50000 [====

In [14]:
ae.save('./AE_models/%s_%s_0.h5'%(id_name,id_model))
ae.save_weights('./AE_models/%s_%s_weights_0.h5'%(id_name,id_model))

In [17]:
ae.load_weights('./AE_models/%s_%s_weights.h5'%(id_name,id_model))

In [ ]:
for i in range(3):
    xx = ae.predict(x_test[i].reshape(1,32,32,3), batch_size=batch_size)
    # print(xx.shape)
    x1 = xx.reshape(32,32,3)
    plt.imshow(x_test[i])
    plt.show()
    plt.imshow(x1)
    plt.show()

In [18]:
def mark(x_1, x_2):
    diff = np.abs(x_1 - x_2)
    marks = np.mean(np.power(diff, 2), axis=(1,2,3))
    return marks

In [19]:
sc_id = []
for i in range(len(x_test)):
    xx = ae.predict(x_test[i].reshape(1,32,32,3))
    score = mark(x_test[i],xx)
    # print(score)
    sc_id.append(-score)

In [20]:
def combine_inliners_outliers(inliers, outliers, i_label=1, o_label=0, verbose=False):

    temp_outliers = outliers
    temp_inliers  = inliers
    if i_label==1:
        i_labels = np.ones(temp_inliers.shape[0])
    else:
        i_labels = np.zeros(temp_inliers.shape[0])
        
    if o_label==0:
        o_labels = np.zeros(temp_outliers.shape[0])
    else:
        o_labels = np.ones(temp_outliers.shape[0])       
              
    mixed_labels =  np.append(i_labels, o_labels)
    mixed_data = np.vstack((temp_inliers, temp_outliers))

    return mixed_data, mixed_labels

In [21]:
#layer_inx specifies the index of the OODL found by "find_oodl" jupyter file.
REM_TOP_LAYER = -2
NUM_CLASS = 10
if id_name=="MNIST" and id_model=="LeNet":
    layer_inx = 0
    OOD_DS_LIST       = OOD_DS_LIST_MNIST
    #**********************************************************
elif id_name=="CIFAR10":
    OOD_DS_LIST      = OOD_DS_LIST_CIFAR10
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 20
    #**********************************************************
elif id_name=="SVHN":
    OOD_DS_LIST      = OOD_DS_LIST_SVHN
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 24

In [ ]:
for i, ood_ds_name in enumerate(OOD_DS_LIST):
    print(ood_ds_name)

In [ ]:
(_,_),(ood_eva_data,_) = load_dataset('SVHN_CROPPED')
das = preprocess_images(id_name, ood_eva_data, id_model, verbose=True)

In [ ]:
scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])

In [ ]:
print(das[0])

In [22]:
# Vriance模块(OODL大类别 --> 删除同样的feature maps)
test = np.asarray(sc_id)
for i, ood_ds_name in enumerate(OOD_DS_LIST):
    (_,_),(ood_eva_data,_) = load_dataset(ood_ds_name)
    das = preprocess_images(id_name, ood_eva_data, id_model, verbose=True)
    
    scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        xx = ae.predict(das[i].reshape(1,32,32,3))
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood) 
    scores, mixed_labels = combine_inliners_outliers(test, ood) 

    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
#     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ood_ds_name))
#     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
#     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

Preprocessing was done for  CIFAR10 VGG
The results of TINYIMAGENET_RESIZED_32:

tnr_at_95_tpr: 88.1699999991183
detection_acc: 91.58499999908413
AUROC:  96.83788299999999


Preprocessing was done for  CIFAR10 VGG
The results of LSUN_RESIZED:

tnr_at_95_tpr: 95.7899999990421
detection_acc: 95.39499999904604
AUROC:  98.959557


Preprocessing was done for  CIFAR10 VGG
The results of ISUN_PATCHED:

tnr_at_95_tpr: 91.4397759093396
detection_acc: 93.2198879541948
AUROC:  97.63841344537816


Preprocessing was done for  CIFAR10 VGG
The results of SVHN_CROPPED:

tnr_at_95_tpr: 0.0076828518745863435
detection_acc: 47.50384142546229
AUROC:  2.622107790411801


Preprocessing was done for  CIFAR10 VGG
The results of G_255:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  100.0


Preprocessing was done for  CIFAR10 VGG
The results of U_255:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  100.0




In [20]:
ood_ds_name = 'foolingimages'

das = np.load('adv_datasets/%s_%s_fooling_images.npy'%(id_name,id_model))

# scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])

sc_ood = []
for i in range(len(das)):
        xx = ae.predict(das[i].reshape(1,32,32,3))
        score = mark(das[i],xx)
        sc_ood.append(-score)

ood = np.asarray(sc_ood) 
scores, mixed_labels = combine_inliners_outliers(test, ood) 

# print('scores is:', (scores[10000:20000]))
fpr, tpr = nums(scores, mixed_labels)
roc_auc = auc(fpr, tpr)
lens = id_eva_data_processed.shape[0]
FP,TN,TP,FN = ErrorRateAt95Recall1(lens,scores, mixed_labels)
ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
print('The results of %s:\n' %(ood_ds_name))
# print(get_summary_statistics(scores, mixed_labels))
# print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
# print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
print("AUROC: ",ROC*100)
print("\n")

The results of foolingimages:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  100.0




In [23]:
x_test = np.load('./advs_new/%s_%s_normal.npy'%(id_name, id_model),allow_pickle=True)
scaler.fit(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test_0 = scaler.transform(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test = x_test_0.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],x_test.shape[3])
print(x_test.shape)

(9347, 32, 32, 3)


In [24]:
sc_id = []
for i in range(len(x_test)):
    xx = ae.predict(x_test[i].reshape(1,32,32,3))
    score = mark(x_test[i],xx)
    # print(score)
    sc_id.append(-score)

In [25]:
test = np.asarray(sc_id)
ADV_DS_LIST = ("fgsm", "bim-a", "bim-b", "jsma", "cw-l2")
for i, ds_name in enumerate(ADV_DS_LIST):
    adv = id_name+"_"+id_model+"_"+ ds_name
    das = np.load('./advs_new/'+adv+'.npy',allow_pickle=True)
    
    scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        xx = ae.predict(das[i].reshape(1,32,32,3))
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood)
    scores, mixed_labels = combine_inliners_outliers(test, ood)
 
    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
    #     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ds_name))
    #     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
    #     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

The results of fgsm:

tnr_at_95_tpr: 77.12215320822266
detection_acc: 86.06107660363632
AUROC:  99.97513117427089


The results of bim-a:

tnr_at_95_tpr: 15.010351966701052
detection_acc: 55.00517598287552
AUROC:  73.31406856838993


The results of bim-b:

tnr_at_95_tpr: 23.015873015608282
detection_acc: 59.00793650732914
AUROC:  84.62939572118616


The results of jsma:

tnr_at_95_tpr: 17.770876466324236
detection_acc: 56.385438232687115
AUROC:  76.5697580634223


The results of cw-l2:

tnr_at_95_tpr: 63.97515527876725
detection_acc: 79.48757763890862
AUROC:  95.81685565215345


